<a href="https://colab.research.google.com/github/JSJeong-me/Machine_Learning/blob/main/ML/10-cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup rendering dependencies for Google Colaboratory.

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Install d3rlpy!

In [2]:
!pip install d3rlpy

     |████████████████████████████████| 1.2 MB 5.2 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 125 kB 64.0 MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=ca3264da5e489cba41b1852122f21bb7a8db4573505fcf4c950432198623d0c5
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


Setup cartpole dataset.

In [3]:
from d3rlpy.datasets import get_cartpole

# get CartPole dataset
dataset, env = get_cartpole()

Setup data-driven deep reinforcement learning algorithm.

In [4]:
from d3rlpy.algos import DiscreteCQL
from d3rlpy.metrics.scorer import discounted_sum_of_advantage_scorer
from d3rlpy.metrics.scorer import evaluate_on_environment
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from sklearn.model_selection import train_test_split

# setup CQL algorithm
cql = DiscreteCQL(use_gpu=False)

# split train and test episodes
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

# start training
cql.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=1,
        scorers={
            'environment': evaluate_on_environment(env), # evaluate with CartPol-v0 environment
            'advantage': discounted_sum_of_advantage_scorer, # smaller is better
            'td_error': td_error_scorer, # smaller is better
            'value_scale': average_value_estimation_scorer # smaller is better
        })

2022-05-01 01:42.53 [debug    ] RoundIterator is selected.
2022-05-01 01:42.53 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20220501014253
2022-05-01 01:42.53 [debug    ] Building models...
2022-05-01 01:42.53 [debug    ] Models have been built.
2022-05-01 01:42.53 [info     ] Parameters are saved to d3rlpy_logs/DiscreteCQL_20220501014253/params.json params={'action_scaler': None, 'alpha': 1.0, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_update_interval': 8000, 'use_gpu': None, 'algorithm': 'DiscreteCQL', 'observation

Epoch 1/1:   0%|          | 0/2487 [00:00<?, ?it/s]

2022-05-01 01:43.18 [info     ] DiscreteCQL_20220501014253: epoch=1 step=2487 epoch=1 metrics={'time_sample_batch': 0.0002580052357770661, 'time_algorithm_update': 0.007473945330077978, 'loss': 0.6861463472888227, 'time_step': 0.007964002496514591, 'environment': 200.0, 'advantage': -2.63836737968158, 'td_error': 1.1459702195786692, 'value_scale': 1.0837596543706949} step=2487
2022-05-01 01:43.18 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20220501014253/model_2487.pt


[(1,
  {'advantage': -2.63836737968158,
   'environment': 200.0,
   'loss': 0.6861463472888227,
   'td_error': 1.1459702195786692,
   'time_algorithm_update': 0.007473945330077978,
   'time_sample_batch': 0.0002580052357770661,
   'time_step': 0.007964002496514591,
   'value_scale': 1.0837596543706949})]

Setup rendering utilities for Google Colaboratory.

In [5]:
import glob
import io
import base64

from gym.wrappers import Monitor
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

# start virtual display
display = Display(visible=0, size=(1400, 900))
display.start()

# play recorded video
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''
            <video alt="test" autoplay loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

Record video!

In [6]:
# wrap Monitor wrapper
env = Monitor(env, './video', force=True)

# evaluate
evaluate_on_environment(env)(cql)

200.0

Let's see how it works!

In [7]:
show_video()